In [ ]:
#pip install openmeteo-requests
#pip install requests-cache retry-requests numpy pandas

In [2]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)




Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  temperature_2m
0   2024-08-20 00:00:00+00:00       14.595500
1   2024-08-20 01:00:00+00:00       14.095500
2   2024-08-20 02:00:00+00:00       13.745500
3   2024-08-20 03:00:00+00:00       13.195499
4   2024-08-20 04:00:00+00:00       12.895500
..                        ...             ...
163 2024-08-26 19:00:00+00:00       18.419500
164 2024-08-26 20:00:00+00:00       16.919500
165 2024-08-26 21:00:00+00:00       15.669499
166 2024-08-26 22:00:00+00:00       14.819500
167 2024-08-26 23:00:00+00:00       14.269500

[168 rows x 2 columns]


In [28]:
from geopy.geocoders import Nominatim

def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Example usage:
location_name = "Pune"
coordinates = get_coordinates(location_name)
if coordinates:
    print(f"Coordinates of {location_name}: {coordinates}")
else:
    print("Location not found.")


Coordinates of Pune: (18.5213738, 73.8545071)


In [29]:
params = {
	"latitude": 18.5213738,
	"longitude": 73.854,
	"hourly": ["rain", "visibility", "wind_speed_180m", "temperature_180m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_rain = hourly.Variables(0).ValuesAsNumpy()
hourly_visibility = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_180m = hourly.Variables(2).ValuesAsNumpy()
hourly_temperature_180m = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["rain"] = hourly_rain
hourly_data["visibility"] = hourly_visibility
hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
hourly_data["temperature_180m"] = hourly_temperature_180m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 18.625°N 74.0°E
Elevation 540.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  rain  visibility  wind_speed_180m  \
0   2024-08-20 00:00:00+00:00   0.0     24140.0         4.693826   
1   2024-08-20 01:00:00+00:00   0.0     24140.0         5.351785   
2   2024-08-20 02:00:00+00:00   0.0     24140.0         3.600000   
3   2024-08-20 03:00:00+00:00   0.0     24140.0         2.595997   
4   2024-08-20 04:00:00+00:00   0.0     24140.0         2.305125   
..                        ...   ...         ...              ...   
163 2024-08-26 19:00:00+00:00   0.0      4320.0        29.354902   
164 2024-08-26 20:00:00+00:00   0.0      8360.0        30.018473   
165 2024-08-26 21:00:00+00:00   0.0     12380.0        30.351225   
166 2024-08-26 22:00:00+00:00   0.0      8300.0        30.018473   
167 2024-08-26 23:00:00+00:00   0.0      4200.0        29.827904   

     temperature_180m  
0           22.960001  
1           22.860001  
2        

In [18]:
import os

In [30]:
#hourly_dataframe.to_csv('flight_data.csv',index=False)
averages=hourly_dataframe.mean()
print(averages)
df=pd.DataFrame([averages],columns=averages.index)

# File path
file_path = 'flight_data.csv'

# Check if the file already exists
file_exists = os.path.isfile(file_path)

df.to_csv('flight_data.csv',mode='a',header=not file_exists,index=False)


date                2024-08-23 11:30:00+00:00
rain                                   0.0125
visibility                       14156.428711
wind_speed_180m                     18.064409
temperature_180m                    23.808216
dtype: object
